### Practical sheet 4 (Experimental Setup)

#### 00-Libraries

In [1]:
##########################################################
# Import Libraries to start work
##########################################################
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

#### 01-Data & Transformation

In [2]:
df = pd.read_csv("banknote.csv",names=["Variance","Skewness","Curtosis","Entropy","Class"])
df.head()

,Variance,Skewness,Curtosis,Entropy,Class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [4]:
X = df[['Variance', 'Skewness', 'Curtosis', 'Entropy']].values
y = df['Class'].values

In [ ]:


#########################################################################
#   Read, Normalize and Split Data
#########################################################################


with open('banknote.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader) # to skip the header file
    X = []
    y = []
    for row in csv_reader:
        y.append(int(float(row[0]) == 1))
        temp= [float(i) for i in row[1:]]
        X.append(temp)

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:

X = np.asarray(X)
y = np.asarray(y)

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
X = np.append(X, np.ones((X.shape[0],1), np.float64), axis=1)


In [9]:
print(X)

[[0.76900389 0.83964273 0.10678269 0.73662766 1.        ]
 [0.83565902 0.82098209 0.12180412 0.64432563 1.        ]
 [0.78662859 0.41664827 0.31060805 0.78695091 1.        ]
 ...
 [0.23738543 0.01176814 0.98560321 0.52475518 1.        ]
 [0.25084193 0.20170105 0.76158701 0.6606745  1.        ]
 [0.32452819 0.49074676 0.34334762 0.88594888 1.        ]]


In [16]:


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234)

In [10]:
#########################################################################
#   Logistic regression
#########################################################################
## incluir os pesos somatorio dos pesos ao quadrado

def compute_loss(y_true, y_pred):
    # binary cross entropy
    epsilon = 1e-9
    y1 = y_true * np.log(y_pred + epsilon)
    y2 = (1-y_true) * np.log(1 - y_pred + epsilon)
    return -np.mean(y1 + y2)

In [ ]:
def _sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def feed_forward(X, weights, bias):
    z = np.dot(X, weights) + bias
    A = _sigmoid(z)
    return A

In [ ]:
def fit(X, y, n_iters, lr):
    n_samples, n_features = X.shape

    # init parameters
    weights = np.zeros(n_features)
    bias = 0
    losses = []

    # gradient descent
    for _ in range(n_iters):
        A = feed_forward(X, weights, bias)
        losses.append(compute_loss(y,A))
        dz = A - y # derivative of sigmoid and bce X.T*(A-y)
        # compute gradients
        dw = (1 / n_samples) * np.dot(X.T, dz)
        db = (1 / n_samples) * np.sum(dz)
        # update parameters
        weights -= lr * dw
        bias -= lr * db
      
    return weights, bias, losses

In [ ]:
def predict(X, weights, bias):
    threshold = .5
    y_hat = np.dot(X, weights) + bias
    y_predicted = _sigmoid(y_hat)
    y_predicted_cls = [1 if i > threshold else 0 for i in y_predicted]
    return np.array(y_predicted_cls)

In [66]:
learning_rate = 0.1
n_iters = 1000
alpha = 10000

In [68]:

weights, bias, losses = fit(X_train, y_train, n_iters, learning_rate,alpha)

In [69]:

predictions = predict(X_test, weights, bias)
print("Test accuracy: {0:.3f}".format(np.sum(np.diag(cm))/np.sum(cm)))


Test accuracy: 0.582


In [ ]:

plt.ion()
plt.figure(1)
plt.plot(range(n_iters), losses, '-g', label='Tensorflow')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.show()
plt.pause(0.01)

In [48]:

cm  = confusion_matrix(np.asarray(y_test), np.asarray(predictions))
print("Confusion Matrix:")
print(np.array(cm))


Confusion Matrix:
[[160   0]
 [115   0]]
